# Análisis exploratorio de la actividad semanal en X

Este notebook realiza un análisis exploratorio de datos (EDA) sobre la actividad de publicación semanal de una cuenta influyente en la red social X (Twitter).

El objetivo es construir una serie temporal agregada por semanas que sirva como base para modelos de predicción del número de tweets en semanas futuras.


In [ ]:
# Clonar el repositorio del proyecto
!git clone https://github.com/0xnito/AP.git

# Entrar al repositorio
%cd AP

# Comprobar la estructura del proyecto
!ls


In [ ]:
# Instalar Kaggle (si no está ya disponible)
!pip install kaggle

In [ ]:
# (Opcional) Subir credenciales de Kaggle para descarga automática
from google.colab import files
files.upload()


In [ ]:
# (Opcional) Configurar Kaggle en Colab
import os

os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)


In [ ]:
import os

os.makedirs("data/raw", exist_ok=True)

csv_path = "data/raw/all_musk_posts.csv"
kaggle_path = "/root/.kaggle/kaggle.json"

if os.path.exists(csv_path):
    print("Dataset ya disponible.")
else:
    if not os.path.exists(kaggle_path):
        print(
            "No se ha encontrado el archivo kaggle.json.\n\n"
            "Para descargar automáticamente los datos:\n"
            "1) Crear un token de Kaggle\n"
            "2) Subir kaggle.json a Colab\n\n"
            "Alternativamente, descarga manualmente el dataset desde:\n"
            "https://www.kaggle.com/datasets/dadalyndell/elon-musk-tweets-2010-to-2025-march\n"
            "y coloca all_musk_posts.csv en data/raw/"
        )
    else:
        print("Token de Kaggle encontrado. Descargando dataset...")
        !kaggle datasets download -d dadalyndell/elon-musk-tweets-2010-to-2025-march -p data/raw
        !unzip -o data/raw/elon-musk-tweets-2010-to-2025-march.zip -d data/raw
        print("Dataset descargado correctamente.")


In [ ]:
import pandas as pd

# Cargar dataset principal (tweets + RTs)
df = pd.read_csv("data/raw/all_musk_posts.csv")

df.head()


In [ ]:
print("Dimensiones del dataset:", df.shape)
df.dtypes


In [ ]:
# Convertir la columna createdAt a datetime
df["createdAt"] = pd.to_datetime(df["createdAt"])

# Crear columna auxiliar sin zona horaria para el análisis temporal
df["createdAt_naive"] = df["createdAt"].dt.tz_localize(None)

# Rango temporal del dataset
df["createdAt"].min(), df["createdAt"].max()


In [ ]:
# Crear columna con la semana (inicio de semana)
df["week"] = df["createdAt_naive"].dt.to_period("W").apply(lambda r: r.start_time)

# Agrupar por semana y contar número de tweets
weekly_counts = (
    df.groupby("week")
      .size()
      .reset_index(name="tweet_count")
)

weekly_counts.head()

In [ ]:
print("Número de semanas:", weekly_counts.shape[0])
weekly_counts["tweet_count"].describe()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.plot(weekly_counts["week"], weekly_counts["tweet_count"])
plt.xlabel("Semana")
plt.ylabel("Número de tweets")
plt.title("Actividad semanal de publicación")
plt.tight_layout()
plt.show()


El análisis exploratorio muestra que la actividad semanal es altamente variable, no estacionaria y con tendencia creciente, lo que justifica el uso de modelos temporales para su predicción.

In [ ]:
# Definición de variables de entrada (X) y salida (y)

# X: número de tweets publicados en la semana actual
# y: número de tweets publicados en la semana siguiente

X = weekly_counts[["tweet_count"]].iloc[:-1]
y = weekly_counts["tweet_count"].iloc[1:]

print("Dimensión de X:", X.shape)
print("Dimensión de y:", y.shape)
